# RAG with CrewAI

### Overview
This notebook demonstrates Retrieval-Augmented Generation (RAG) using CrewAI agents and your DuckDB vector database. Instead of the LLM generating answers from its training data alone, RAG retrieves relevant passages from your database and uses them to ground responses in your specific documents.

## Section 1: Setup and Configuration

### 1.1 Install Required Packages


In [8]:
!pip install crewai duckdb

### 1.2 Configure API Key

In [9]:
from google.colab import userdata
import os

# Get OpenAI API key from Colab secrets
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

# Set as environment variable (CrewAI reads from here)
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

### 1.3 Import Libraries and Set Database Path
- **Agent**: Defines an AI agent with specific role and capabilities
- **Task**: Represents work to be done by an agent
- **Crew**: Orchestrates multiple agents working together
- **LLM**: Language model configuration
- **tool**: Decorator to create tools that agents can use

In [10]:
from crewai import Agent, Task, Crew, LLM
from crewai.tools import tool
import duckdb

# Path to your DuckDB vector database
DB_PATH = "/content/drive/Shareddrives/# MDSC-30801-Language-Processing-in-Practice/Fall 2025/Week 12/Chronicles_VectorDB/chronicles_vector.duckdb"

## Section 2: Creating the Retrieval Tool

## 2.1 Define Database Retrieval Tool
- The Retrieval Step in RAG:
    - Input: User query (e.g., "What themes appear in Chronicles?")
    - Process: Semantic search over vector database
    - Output: Top-k most relevant passages (context)
    - Next: These passages get added to LLM prompt for generation

- `@tool` Decorator:
    - Converts Python function into a CrewAI tool
    - First argument: Tool name (shown to the agent)
    - Docstring: Describes what the tool does (agent reads this!)
    - This tool handles the R in RAG (Retrieval)

- Semantic Search Process:
    - Convert user query to embedding vector (384 dimensions)
    - Compare query embedding to all document embeddings in database
    - Use cosine similarity to measure semantic relevance
    - Return top 10 most similar passages (retrieved context)
    - Agent uses these passages to augment its generation

In [11]:
@tool("Query Database")
def query_db(query: str) -> str:
    """Search the Chronicles database containing Bible Project video content and biblical text.
    This is the RETRIEVAL step in RAG - finding relevant context from your documents.

    Args:
        query: Search query about Chronicles

    Returns:
        Retrieved passages from the database (context for generation)
    """
    try:
        # Connect to database in read-only mode
        # read_only=True prevents accidental modifications
        con = duckdb.connect(DB_PATH, read_only=True)

        # Import embedding model (same one used to create database)

        # MAKE SURE YOU'RE USING THE SAME EMBEDDING MODEL AS YOUR DB!!!!
        from sentence_transformers import SentenceTransformer
        model = SentenceTransformer('all-MiniLM-L6-v2')

        # Convert query text to embedding vector
        query_embedding = model.encode(query).tolist()

        # Search database using cosine similarity
        # Returns top 10 most similar passages

        # UPDATE TABLE NAME WITH YOUR TABLE!!!!!!
        # COULD ADD SIMILARITY THRESHOLD 'WHERE similarity > 0.7'
        results = con.execute("""
            SELECT text, array_cosine_similarity(embedding, ?::FLOAT[384]) as similarity
            FROM chr_rag_documents
            ORDER BY similarity DESC
            LIMIT 10
        """, [query_embedding]).fetchall()

        # Close database connection
        con.close()

        # Format results for the agent
        if results:
            passages = [row[0] for row in results]  # Extract just the text
            # Join passages with separators for readability
            return "\n\n---\n\n".join([f"Passage {i+1}:\n{doc}" for i, doc in enumerate(passages)])
        else:
            return "No relevant passages found."

    except Exception as e:
        return f"Error: {str(e)}"

## Section 3: Creating the RAG Agent

## 3.1 Configure LLM and Create Agent
Agent Configuration for RAG:
- `role`:
    - Defines agent's identity
    - "Content Assistant" → focuses on answering with retrieved sources
    - Influences how it uses retrieved context

- `goal`:
    - "Answer questions using the database" = RAG objective
    - Explicitly tells agent to ground answers in retrieved passages

- `backstory`:
    - Provides context about capabilities
    - Emphasizes database access (retrieval capability)

- `tools`:
    - `query_chronicles_db`: The retrieval function (R in RAG)
    - Agent decides when to retrieve
    - Can retrieve multiple times with different queries
    - This is what makes it RAG vs pure generation

- `llm`:
    - Language model for generation (G in RAG)
    - `gpt-4o-mini`: Good balance for RAG tasks
    - Takes retrieved context + question → generates answer

- `verbose=True`:
    - Prints the entire RAG pipeline
    - Essential for understanding and debugging RAG

- `max_iter=3`:
    - CRITICAL: Limits retrieval calls
    - Prevents infinite retrieval loops
    - RAG best practice: 1-3 retrievals per question
    - Cost control for API calls

In [12]:
# Configure the language model for GENERATION step in RAG
# gpt-4o-mini: Fast, cost-effective model for generating from retrieved context
llm = LLM(model="gpt-4o-mini")

# Create the Chronicles RAG agent
rag_agent = Agent(
    role='Chronicles Content Assistant',                  # Agent's job title
    goal='Answer questions about Chronicles using the database',  # Primary objective: RAG
    backstory='You are an expert who has access to a database with content about the book of Chronicles.',  # Context
    tools=[query_db],                          # Retrieval tool (R in RAG)
    llm=llm,                                              # Generation model (G in RAG)
    verbose=True,                                         # Print RAG pipeline steps
    max_iter=3                                            # Limit retrieval calls
)

## Section 4: RAG Pipeline Execution

### 4.1 Create RAG Query Function
- `Task`:
    - Represents the RAG question-answering task
    - `description`: User's question (what needs retrieval + generation)
    - `agent`: Which agent runs the RAG pipeline
    - `expected_output`: Answer grounded in retrieved documents

- `Crew`:
    - Orchestrates the RAG workflow
    - Manages retrieval calls and generation
    - Handles coordination for multi-agent RAG (if expanded)

- `max_rpm=20`:
    - Rate limit: 20 API requests per minute
    - Important for RAG: each retrieval + generation = 2+ API calls
    - Prevents hitting OpenAI rate limits
    - Protects against excessive costs in RAG applications

- `crew.kickoff()`:
    - Starts the RAG pipeline execution
    - Agent begins: analyze question → retrieve → generate
    - Returns final generated answer with retrieved context

In [13]:
def ask_question(question: str):
    """
    Execute RAG pipeline: Retrieve relevant passages, then generate answer.

    Args:
        question: User's question about Chronicles

    Returns:
        Generated answer grounded in retrieved passages
    """
    # Create a task for the agent
    task = Task(
        description=f'{question}',  # The user's question
        agent=rag_agent,      # RAG agent handles this
        expected_output='A comprehensive answer based on the database content'  # Grounded in retrieved context
    )

    # Create a crew to orchestrate the RAG pipeline
    crew = Crew(
        agents=[rag_agent],   # List of agents (single-agent RAG here)
        tasks=[task],                # List of tasks
        verbose=True,                # Print RAG pipeline execution
        max_rpm=20                   # Rate limit: 20 requests per minute
    )

    try:
        # Execute RAG pipeline: retrieve → augment → generate
        result = crew.kickoff()
        return result
    except Exception as e:
        return f"Error: {str(e)}"

### 4.2 Test the RAG System


In [18]:
# Ask a question about Chronicles
question = "What are some of the major issues concerning the book of Chronicles? Can you cite some of the sources from my database?"

# Execute RAG pipeline
answer = ask_question(question)

# Display the generated answer (grounded in retrieved passages)
print(answer)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: be973830-3652-4853-ad76-41daee08b18d                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Chronicles Content Assistant                                                                            │
│                                                                                                                 │
│  Task: What are some of the major issues concerning the book of Chronicles? Can you cite some of the sources    │
│  from my database?                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Chronicles Content Assistant                                                                            │
│                                                                                                                 │
│  Thought: Action: Query Database                                                                                │
│                                                                                                                 │
│  Using Tool: Query Database                                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Chronicles Content Assistant                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The Book of Chronicles presents several major issues and themes that have been discussed in scholarly          │
│  literature. Here's a comprehensive look at these issues based on the retrieved passages:                       │
│                                                                                                                 │
│  1. **Distinct Purpose and Context**: Chronicles offers a narrative of Israel's history, specifically aimed at  │
│  encouraging the postexilic community. It emphasizes the significance of the temple and its role in             │
│  maintaining Israel's identity and direction. The narrative is not simply a repetition of Kings but is          │
│  restructured to provide hope and inspiration to a chastened community (Observation: Passage 2).                │
│                                                                                                                 │
│  2. **Literary Structure and Sources**: The Chronicler's engagement with sources is a critical issue. While     │
│  Chronicles rewrites much of Israel's history found in the books of Samuel and Kings, it does so selectively,   │
│  adding new material and reorganizing existing content. This raises questions on whether Chronicles intends to  │
│  supplement or supplant the established narratives (Observation: Passage 5). Scholars note that the literary    │
│  efforts of the Chronicler often go unconsidered, leading to discussions on its unique structure (Observation:  │
│  Passages 1 and 6).                                                                                             │
│                                                                                                                 │
│  3. **Theological and Political Ideology**: Critics suggest that Chronicles is a theological and political      │
│  work that aims to reclaim Israel's historical narrative against competing ideologies, ensuring that it         │
│  reflects a priestly perspective rather than a prophetic one (Observation: Passages 3 and 4). The absence of    │
│  certain traditional narratives (like the Exodus) also contributes to this ideological reworking.               │
│                                                                                                                 │
│  4. **Prophecy and Its Functions**: The role and nature of prophecy in Chronicles have been increasingly        │
│  studied. Issues involve the forms, functions, and implications of prophetic narratives, which demonstrate the  │
│  interaction between the prophetic tradition and the Davidic ideology present in the text (Observation:         │
│  Passage 8).                                                                                                    │
│                                                                                                                 │
│  5. **Interpretative Complexities**: Scholars debate whether Chronicles is merely a commentary on earlier       │
│  texts or if it constitutes an original historiographical endeavor. Many argue that to comprehend Chronicles    │
│  fully, one must also understand Samuel and Kings, as it reflects a sophisticated reinterpretation rather than  │
│  a simple retelling (Observation: Passages 9 and 10).                                                           │
│                                                        

The Book of Chronicles presents several major issues and themes that have been discussed in scholarly literature. Here's a comprehensive look at these issues based on the retrieved passages:

1. **Distinct Purpose and Context**: Chronicles offers a narrative of Israel's history, specifically aimed at encouraging the postexilic community. It emphasizes the significance of the temple and its role in maintaining Israel's identity and direction. The narrative is not simply a repetition of Kings but is restructured to provide hope and inspiration to a chastened community (Observation: Passage 2).

2. **Literary Structure and Sources**: The Chronicler's engagement with sources is a critical issue. While Chronicles rewrites much of Israel's history found in the books of Samuel and Kings, it does so selectively, adding new material and reorganizing existing content. This raises questions on whether Chronicles intends to supplement or supplant the established narratives (Observation: Passage 5)

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯